In [13]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
import pickle

In [14]:
data = pd.read_csv('./Churn_Modelling.csv')

In [15]:
## Preprocess the data
## Drop irrelevent Features

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [16]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [17]:
one_encoder = OneHotEncoder()
geo_encoded = one_encoder.fit_transform(data[['Geography']])

In [18]:
geo_encoder_df = pd.DataFrame(geo_encoded.toarray(), columns=one_encoder.get_feature_names_out(['Geography']))

In [19]:
data = pd.concat([data.drop(['Geography'], axis=1), geo_encoder_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [20]:
## ivide the Dataset into independent and dependent features
X = data.drop(['Exited'], axis=1)
y = data['Exited']

## Split in training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_encoder.pkl', 'wb') as file:
    pickle.dump(one_encoder, file)
    
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN PART

In [27]:
# define a function to create a model and try different paarmeters (keras classifier)

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation="relu", input_shape=(X_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [31]:
## Create a keras classifier
model = KerasClassifier(layers=1, neurons=32, build_fn=create_model, verbose=1)

In [32]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2, 3],
    'epochs': [50, 100]
}

In [33]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

c:\Users\amanc\Desktop\tens_ml_project\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/100


250/250 [==============================] - 1s 1ms/step - loss: 0.5055 - accuracy: 0.7675
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4244 - accuracy: 0.8166
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4055 - accuracy: 0.8286
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3889 - accuracy: 0.8364
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3752 - accuracy: 0.8455
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3651 - accuracy: 0.8491
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3578 - accuracy: 0.8520
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3531 - accuracy: 0.8536
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3492 - accuracy: 0.8555
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3466 - ac